In [37]:
from pyspark import SparkConf
from pyspark.sql import SparkSession
from estados import estados_brasileiros
from functools import reduce
import unicodedata

In [38]:
conf = SparkConf()
spark = SparkSession.builder.config(conf=conf).getOrCreate()

In [39]:
dfTweets = spark.read.csv("../twitter/output/tweets_analysis/tweets-analisados-2022-06-06.csv", header=True,
                          inferSchema=True)

In [40]:
dfTweetsTratado = dfTweets.filter(dfTweets.len > 5)

In [41]:
dfTweetsTratado.coalesce(1).write.option("header", True).csv("./tmp/spark_output/datacsv/tweets", emptyValue='')

In [42]:
dfEnem = spark.read.option('delimiter', ';').option("encoding", "ISO-8859-1").csv("./bases/MICRODADOS_ENEM_2019.csv",
                                                                                  header=True, inferSchema=True)

In [43]:
# Criar Sample
# dfEnemSample = dfEnem.sample(0.1, seed=69420)

In [44]:
# dfEnemSample.coalesce(1).write.option("header",True).csv("./tmp/spark_output/datacsv/enem", emptyValue='')

In [45]:
dfEnemEstadosTratados = dfEnem.na.replace(to_replace=estados_brasileiros)

In [46]:
dfEnemTratado = dfEnemEstadosTratados.selectExpr("IN_TREINEIRO as Treineiro",
                                                 "SG_UF_PROVA as Estado",
                                                 "TP_PRESENCA_CN as Presenca_CN",
                                                 "TP_PRESENCA_CH as Presenca_CH",
                                                 "TP_PRESENCA_LC as Presenca_LC",
                                                 "TP_PRESENCA_MT as Presenca_MT",
                                                 "NU_NOTA_CN as Nota_CN",
                                                 "NU_NOTA_CH as Nota_CH",
                                                 "NU_NOTA_LC as Nota_LC",
                                                 "NU_NOTA_MT as Nota_MT",
                                                 "TP_STATUS_REDACAO as Status_Redacao",
                                                 "NU_NOTA_REDACAO as Nota_Redacao")

In [47]:
dfEnemTratadoSample = dfEnemTratado.sample(0.1, seed=69420)

In [48]:
dfEnemTratadoSample.coalesce(1).write.option("header", True).csv("./tmp/spark_output/datacsv/enem", emptyValue='')

In [101]:
dfIDD = spark.read.json("./bases/IDD_2019.json", multiLine=True)
dfIDD = dfIDD.fillna(0) # This is the number 0
dfIDDEstadosTratados = dfIDD.na.replace(to_replace=estados_brasileiros)

In [102]:
colunas = dfIDDEstadosTratados.columns
novas_colunas = list(map(lambda item: unicodedata.normalize('NFKD', item.replace("(", "").replace(")", "").replace("-",
                                                                                                                   "").replace(
    " ", "_").upper()).encode('ascii', 'ignore').decode('utf-8', 'ignore'), colunas))
dfIDDComNovasColunas = reduce(lambda data, idx: data.withColumnRenamed(colunas[idx], novas_colunas[idx]),
                              range(len(colunas)), dfIDDEstadosTratados)

In [103]:
dfIDDTratado = dfIDDComNovasColunas.selectExpr("IDD_CONTINUO as IDD_Continuo",
                                                           "IDD_FAIXA as IDD_Faixa",
                                                           "NOTA_BRUTA__IDD as Nota_Bruta_IDD",
                                                           "SIGLA_DA_UF as Estado_Curso",
                                                           "NOME_DA_IES as Instituicao_Ensino",
                                                           "No_DE_CONCLUINTES_INSCRITOS as Concluintes_Inscritos",
                                                           "No_DE_CONCLUINTES_PARTICIPANTES as Concluintes_Participantes",
                                                           "No_DE_CONCLUINTES_PARTICIPANTES_COM_NOTA_NO_ENEM as Concluintes_Participantes_Nota_Enem",
                                                           "PROPORCAO_DE_CONCLUINTES_PARTICIPANTES_COM_NOTA_NO_ENEM as Prop_Concluintes_Participantes_Nota_Enem_IDD",
                                                           "AREA_DE_AVALIACAO as Curso")

In [105]:
dfIDDTratado.coalesce(1).write.option("header", True).csv("./tmp/spark_output/datacsv/idd", emptyValue='')
